In [6]:
import fbapi
import matplotlib.pyplot as plt
import os
import re
import pandas as pd

In [7]:
NAME_FILE = "names_storage.json"

In [8]:
ids = pd.Series(map(lambda x: re.match(r'\d+', x)[0], os.listdir('./log/')), dtype='int64').drop_duplicates()

In [9]:
if os.path.exists(NAME_FILE):
    known = pd.read_json(NAME_FILE)
else:
    known = pd.DataFrame(columns=['id', 'name'], dtype=['int64', 'string'])

known.head()

,id,name
0,100001558699342,PiotrunioMode
1,100002952972817,emilia.magdziak
10,100002858335011,mvksymilian
100,100002730266263,MeWashKier
101,100001369462026,aleksander.augustyniak.7


In [10]:
ids = pd.DataFrame(ids).rename(columns={0: 'id'})
unlabeled = pd.merge(ids, known, how='outer', on='id', indicator=True).query('_merge == "left_only"').drop(columns=['_merge']).reset_index()
unlabeled.head()

,index,id,name
0,117,100006463587706,NaN


In [11]:
friends = len(unlabeled)
for i in range(friends):
    t_name = fbapi.get_user_name(unlabeled.loc[i, 'id'])
    print(i, "/", friends, "-", t_name)
    unlabeled.loc[i, 'name'] = t_name

0 / 1 - agata.dominiak.77


In [12]:
unlabeled.head()

,index,id,name
0,117,100006463587706,agata.dominiak.77


In [13]:
all_names = unlabeled.set_index(unlabeled['index']).drop(columns=['index'])
all_names = pd.merge(known, all_names, how='outer', on='id')
all_names['name'] = all_names['name_x'].fillna(all_names['name_y'])
all_names = all_names.drop(columns=['name_x', 'name_y'])
all_names.head()

,id,name
0,100001558699342,PiotrunioMode
1,100002952972817,emilia.magdziak
2,100002858335011,mvksymilian
3,100002730266263,MeWashKier
4,100001369462026,aleksander.augustyniak.7


In [14]:
all_names.to_json(NAME_FILE)